In [3]:
import cv2
import face_recognition
 
imgAng = face_recognition.load_image_file('andrew_ng.jpg')
Test = face_recognition.load_image_file('ian_godfellow.jpg')
 
fLoc = face_recognition.face_locations(imgAng)[0]
encodeAng = face_recognition.face_encodings(imgAng)[0]

fLocTest = face_recognition.face_locations(Test)[0]
encTest = face_recognition.face_encodings(Test)[0]

result = face_recognition.compare_faces([encodeAng],encTest)
faceDist = face_recognition.face_distance([encodeAng],encTest)

print(result,faceDist)
 

[False] [0.6767912]


In [7]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
# from PIL import ImageGrab
 
pathlib = 'ImagesAttendance'
images = []
Names = []
myList = os.listdir(pathlib)
print(myList)
for cl in myList:
    currImg = cv2.imread(f'{pathlib}/{cl}')
    images.append(currImg)
    Names.append(os.path.splitext(cl)[0])
print(Names)
 
def DbEncodings(images):
    encList = []
    for image in images:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        enc = face_recognition.face_encodings(image)[0]
        encList.append(enc)
    return encList
 
def Attendance(name):
    with open('Attendance_Register.csv','r+') as f:
        DataList = f.readlines()
        names = []
        for data in DataList:
            ent = data.split(',')
            names.append(ent[0])
        if name not in names:
            curr = datetime.now()
            dt = curr.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dt}')

#### FOR CAPTURING SCREEN RATHER THAN WEBCAM
# def captureScreen(bbox=(300,300,690+300,530+300)):
#     capScr = np.array(ImageGrab.grab(bbox))
#     capScr = cv2.cvtColor(capScr, cv2.COLOR_RGB2BGR)
#     return capScr
 
encodeKnown = DbEncodings(images)
print('Encoding Complete')
 
cap = cv2.VideoCapture(0)
 
while True:
    _, img = cap.read()
    image = cv2.resize(img,(0,0),None,0.25,0.25)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    facesInFrame = face_recognition.face_locations(image)
    encodesInFrame = face_recognition.face_encodings(image,facesInFrame)

    for encFace,faceLoc in zip(encodesInFrame,facesInFrame):
        matchList = face_recognition.compare_faces(encodeKnown,encFace)
        faceDis = face_recognition.face_distance(encodeKnown,encFace)
        #print(faceDis)
        match = np.argmin(faceDis)

        if matchList[match]:
            name = Names[match].upper()
            #print(name)
            y,w,z,x = faceLoc
            y, w, z, x1 = y*4,w*4,z*4,x*4
            cv2.rectangle(img,(x,y),(w,z),(0,255,0),2)
            cv2.rectangle(img,(x,z-35),(w,z),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x+6,z-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            Attendance(name)

    cv2.imshow('Webcam',img)
    k = cv2.waitKey(0)
    if k == 27:         # wait for ESC key to exit
        break
        cv2.destroyAllWindows()

['andrew_ng.jpg', 'ian_godfellow.jpg', 'Jayita.jpg']
['andrew_ng', 'ian_godfellow', 'Jayita']
Encoding Complete


KeyboardInterrupt: 